Description:

This script aims to test all carbon source media defined by Ecocyc (https://ecocyc.org/ECOLI/new-image?object=Growth-Media): individual growth media (the ones with data), microarray Plate ID: Biolog PM1 and PM2. This script will use the basal simulation after cp2 as the base, and run FBA external to the model to see if model solves and see if there is growth (needs definition) and analyze gene usage. 

In [1]:
import numpy as np
import ast
import pandas as pd
import os
import matplotlib.pyplot as plt
import dill
import requests
import xmltodict
import cvxpy as cp
import itertools
import networkx as nx
import plotly.express as px

from scipy.special import logsumexp

%matplotlib inline
# sns.set(style='dar|kgrid', palette='viridis', context='talk')

os.chdir(os.path.expanduser('~/dev/vivarium-ecoli'))

from ecoli.processes.metabolism_redux_classic import NetworkFlowModel, FlowResult
from ecoli.library.schema import numpy_schema, bulk_name_to_idx, listener_schema, counts
from ecoli.processes.registries import topology_registry
TOPOLOGY = topology_registry.access("ecoli-metabolism-redux")
import plotly.express as px

%load_ext autoreload

In [2]:
# load checkpoint 2 model
time = '400'
date = '2025-05-15'
experiment = 'NEW_NewGenes_checkpoint2'
condition = 'basal'
entry = f'{experiment}_{time}_{date}'
folder = f'out/metabolism-comparison/{condition}/{entry}/'

output = np.load(folder + '0_output.npy',allow_pickle='TRUE').item()
# output = np.load(r"out/geneRxnVerifData/output_glc.npy", allow_pickle=True, encoding='ASCII').tolist()
output = output['agents']['0']
fba = output['listeners']['fba_results']
bulk = pd.DataFrame(output['bulk'])
f = open(folder + 'agent_steps.pkl', 'rb')
agent = dill.load(f)
f.close()

In [3]:
# get commonly stored variables
metabolism = agent['ecoli-metabolism-redux-classic']
stoichiometry = metabolism.stoichiometry.copy()
reaction_names = metabolism.reaction_names
fba_new_reaction_ids = metabolism.parameters["fba_new_reaction_ids"]
fba_reaction_ids_to_base_reaction_ids = metabolism.parameters['fba_reaction_ids_to_base_reaction_ids']
metabolites = metabolism.metabolite_names.copy()
binary_kinetic_idx = metabolism.binary_kinetic_idx
exchange_molecules = metabolism.exchange_molecules

S = stoichiometry .copy()
S = pd.DataFrame(S, index=metabolites , columns=reaction_names )
homeostatic_count = pd.DataFrame(fba["homeostatic_metabolite_counts"], columns=metabolism.homeostatic_metabolites).loc[24, :]
homeostatic = pd.DataFrame(fba["target_homeostatic_dmdt"], columns=metabolism.homeostatic_metabolites).loc[24, :]
maintenance = pd.DataFrame(fba["maintenance_target"][1:], columns=['maintenance_reaction']).iat[24, 0]
kinetic = pd.DataFrame(fba["target_kinetic_fluxes"], columns=metabolism.kinetic_constraint_reactions).loc[24, :].copy()

In [4]:
# parameters that are the same across the two simulation
kinetic_reaction_ids = metabolism.kinetic_constraint_reactions
allowed_exchange_uptake = metabolism.allowed_exchange_uptake
FREE_RXNS = ["TRANS-RXN-145", "TRANS-RXN0-545", "TRANS-RXN0-474"]
ADDED_RXNS = ['HS-TRANSPORT-RXN-CPD0-1202', 'HS-TRANSPORT-RXN-CPD0-1202 (reverse)',
                   'HS-TRANSPORT-RXN[CCO-OUTER-MEM]-OXAMATE', 'HS-TRANSPORT-RXN[CCO-OUTER-MEM]-OXAMATE (reverse)',
                   'HS-TRANSPORT-RXN[CCO-PM-BAC-NEG]-OXAMATE', 'HS-TRANSPORT-RXN[CCO-PM-BAC-NEG]-OXAMATE (reverse)',
                   'HS-BETA-GLUCURONID-RXN_CPD-3611//METOH', 'HS-SPONTANEOUS-TRANSPORT[CCO-OUTER-MEM]-HCN', 'HS-SPONTANEOUS-TRANSPORT[CCO-OUTER-MEM]-HCN (reverse)',
                   'HS-SPONTANEOUS-TRANSPORT[CCO-PM-BAC-NEG]-HCN','HS-SPONTANEOUS-TRANSPORT[CCO-PM-BAC-NEG]-HCN (reverse)']

# Define functions for testing

In [5]:
def get_subset_S(S, met_of_interest):
    S_met = S.loc[met_of_interest, :]
    S_met = S_met.loc[:,~np.all(S_met == 0, axis=0)]
    return S_met, S_met.columns

def get_keys(dict, value):
    return [key for key in dict if dict[key] == value]

def test_NetworkFlowModel(objective_weights,
                          uptake_addition = set([]), uptake_removal = set([]), new_exchange_molecules = set([]),
                          add_metabolite = None, add_reaction = None, add_kinetic = None, remove_reaction = None, force_reaction = None, solver_choice=cp.GLOP):
    # update exchanges
    uptake = metabolism.allowed_exchange_uptake.copy()
    uptake = set(uptake)
    uptake = uptake | uptake_addition
    uptake = uptake - uptake_removal
    
    exchange_molecules = metabolism.exchange_molecules.copy()
    exchange_molecules = exchange_molecules | new_exchange_molecules
    
    # update stoichiometry
    reaction_names = metabolism.reaction_names.copy()
    kinetic_reaction_ids = metabolism.kinetic_constraint_reactions.copy()
    kinetic = pd.DataFrame(fba["target_kinetic_fluxes"], columns=metabolism.kinetic_constraint_reactions).loc[24, :].copy()
    metabolites = metabolism.metabolite_names.copy()
    
    S_new = stoichiometry.copy()
    
    if add_metabolite is not None: # add to metabolites list because they are currently not included in the model
        for m in add_metabolite:
            if m not in metabolites:
                metabolites.append(m)
        # append rows of zeros to S_new of length add_metabolite
        S_new = np.concatenate((S_new, np.zeros((len(add_metabolite), S_new.shape[1]))), axis=0)
    
    if add_reaction is not None:
        # assert add_reaction is a dictionary
        assert isinstance(add_reaction, dict)
        
        for r,s in add_reaction.items():
            if r not in reaction_names:
                reaction_names.append(r)
            # append columns of reaction stoich to S_new of length add_reaction
            new_reaction = np.zeros((S_new.shape[0], 1))
            for m, v in s.items():
                new_reaction[metabolites.index(m), 0] = v
            S_new = np.concatenate((S_new, new_reaction), axis=1)
    
    if add_kinetic is not None:
        # assert add_kinetic is a dictionary
        assert isinstance(add_kinetic, dict)
        
        for r, v in add_kinetic.items():
            if r not in kinetic_reaction_ids:
                kinetic_reaction_ids.append(r)
                kinetic[r] = v
                
    if remove_reaction is not None:
        for r in remove_reaction:
            r_idx = reaction_names.index(r)
            S_new = np.delete(S_new, r_idx, axis=1)
            reaction_names.remove(r)
            if r in kinetic_reaction_ids:
                kinetic_reaction_ids.remove(r)
                del kinetic[r]
                
    if force_reaction is not None:
        force_reaction_idx = np.array([reaction_names.index(r) for r in force_reaction])
    else:
        force_reaction_idx = force_reaction

    # Solve NetworkFlowModel
    model = NetworkFlowModel(
            stoich_arr=S_new,
            metabolites=metabolites,
            reactions=reaction_names,
            homeostatic_metabolites=metabolism.homeostatic_metabolites,
            kinetic_reactions=kinetic_reaction_ids,
            free_reactions=FREE_RXNS)
    model.set_up_exchanges(exchanges=exchange_molecules, uptakes=uptake)
    solution: FlowResult = model.solve(
            homeostatic_concs=homeostatic_count * metabolism.counts_to_molar.asNumber(), # in conc
            homeostatic_dm_targets=np.array(list(dict(homeostatic).values())), # *10^7
            maintenance_target=maintenance, # *10^6 ish
            kinetic_targets=np.array(list(dict(kinetic).values())), # *10^6 ish
            # binary_kinetic_idx=binary_kinetic_idx, #7646
            binary_kinetic_idx=None,
            force_flow_idx=force_reaction_idx,
            objective_weights=objective_weights, #same
            upper_flux_bound= 1000000000, # increase to 10^9 because notebook runs FlowResult using Counts, WC runs using conc.   
            solver=solver_choice) #SCS. ECOS, MOSEK
    return solution.objective, solution.velocities, reaction_names, S_new, metabolites, kinetic

# Create Dataframes to Store Results

In [6]:
# all reactions
sim = pd.DataFrame(fba["estimated_fluxes"], columns= reaction_names).mean(axis=0).copy()

# kinetic reactions
kc_target_cp2 = pd.DataFrame(fba["target_kinetic_fluxes"], columns= kinetic_reaction_ids).mean(axis=0).copy()

In [7]:
df_all = sim.copy()
df_all.columns = ['sim_cp2_basal']

In [8]:
df_all

1-ACYLGLYCEROL-3-P-ACYLTRANSFER-RXN                                                                               0.000000e+00
1.1.1.127-RXN                                                                                                     0.000000e+00
1.1.1.127-RXN (reverse)                                                                                           0.000000e+00
1.1.1.215-RXN (reverse)                                                                                           0.000000e+00
1.1.1.251-RXN                                                                                                     0.000000e+00
                                                                                                                      ...     
XYLULOKIN-RXN-CPD-24961/ATP//XYLULOSE-5-PHOSPHATE/ADP/PROTON.47.                                                  0.000000e+00
YIAE1-RXN (reverse)                                                                                            

# Network Flow New Enviornments

In [9]:
# microarray plate 1: ~ tests 96 carbon sources
conditions = {
    'A1 - Carbon Negative Control - MIX0-80': {
        'Add': set([]),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'A2 - L-Arabinose* - MIX0-420': {
        'Add': set(['L-ARABINOSE[p]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'A3 - N-Acetyl-D- Glucosamine* - MIX0-421': {
        'Add': set(['N-acetyl-D-glucosamine[p]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'A4 - D-Saccharic acid - MIX0-422': {
        'Add': set(['D-GLUCARATE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'A5 - Succinic acid - MIX0-423': {
        'Add': set(['SUC[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'A6 - D-Galactose* - MIX0-424': {
        'Add': set(['GALACTOSE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'A7 - L-Aspartic acid - MIX0-425 ': {
        'Add': set(['L-ASPARTATE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'A8 - L-Proline - MIX0-426': {
        'Add': set(['PRO[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'A9 - D-Alanine - MIX0-427': {
        'Add': set(['D-ALANINE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'A10 - D-Trehalose - MIX0-428': {
        'Add': set(['TREHALOSE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'A11 - D-Mannose - MIX0-429': {
        'Add': set(['CPD-13559[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'A12 - Dulcitol - MIX0-430': {
        'Add': set(['GALACTITOL[p]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'B1 - D-Serine - MIX0-431': {
        'Add': set(['D-SERINE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    }, 
    'B2 - D-Sorbitol - MIX0-432': {
        'Add': set(['SORBITOL[e]']),
        'Remove': set(['GLC[p]','CA+2[p]']),
    },
    'B3 - Glycerol - MIX0-433': {
        'Add': set(['GLYCEROL[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'B4 - L-Fucose* - MIX0-434': {
        'Add': set(['L-fucoses[p]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    }, 
    'B5 - D-Glucuronic acid* - MIX0-435': {
        'Add': set(['CPD-15530[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    }, 
    'B6 - D-Gluconic acid - MIX0-436': {
        'Add': set(['GLUCONATE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'B7 - DL-α- Glycerol Phosphate - MIX0-437': {
        'Add': set(['GLYCEROL-3P[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    }, 
    'B8 - D-Xylose* - MIX0-438': {
        'Add': set(['CPD-15377[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'B9 - L-Lactic acid - MIX0-439': {
        'Add': set(['L-LACTATE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    }, 
    'B10 - Formic acid - MIX0-440': {
        'Add': set(['FORMATE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'B11 - D-Mannitol - MIX0-441': {
        'Add': set(['MANNITOL[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'B12 - L-Glutamic acid - MIX0-442': {
        'Add': set(['GLT[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'C1 - D-Glucose- 6-Phosphate* - MIX0-443': {
        'Add': set(['GLC-6-P[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'C2 - D-Galactonic acid-γ- Lactone - MIX0-444': {
        'Add': set(['D-GALACTONO-1-4-LACTONE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'C3 - DL-Malic acid - MIX0-445': {
        'Add': set(['MAL[e]', 'CPD-660[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'C4 - D-Ribose* - MIX0-446': {
        'Add': set(['CPD0-1110[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'C5 - Tween 20 - MIX0-797' : None, # not in the model
    'C6 - L-Rhamnose* - MIX0-447': {
        'Add': set(['RHAMNOSE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'C7 - D-Fructose - MIX0-448': {
        'Add': set(['BETA-D-FRUCTOSE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'C8 - Acetic acid - MIX0-449': {
        'Add': set(['ACET[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'C9 - α-D-Glucose - MIX0-450': {
        'Add': set(['ALPHA-GLUCOSE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'C10 - Maltose - MIX0-451': {
        'Add': set(['MALTOSE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'C11 - D-Melibiose - MIX0-452': {
        'Add': set(['MELIBIOSE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'C12 - Thymidine - MIX0-453': {
        'Add': set(['THYMIDINE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'D1 - L-Asparagine - MIX0-454': {
        'Add': set(['ASN[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'D2 - D-Aspartic acid - MIX0-455': {
        'Add': set(['CPD-302[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'D3 - D-Glucosaminic acid - MIX0-456': {
        'Add': set(['GLUCOSAMINATE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'D4 - 1,2-Propanediol - MIX0-457': {
        'Add': set(['PROPANE-1-2-DIOL[c]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'D5 - Tween 40 - MIX0-798': None, # not in the model
    'D6 - α-Ketoglutaric acid - MIX0-458': {
        'Add': set(['2-KETOGLUTARATE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'D7 - α-Ketobutyric acid - MIX0-459': {
        'Add': set(['2-OXOBUTANOATE[p]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'D8 - α-Methyl-D- Galactoside - MIX0-786': {
        'Add': set(['CPD-3565[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'D9 - α-D-Lactose - MIX0-460': {
        'Add': set(['Alpha-lactose[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'D10 - Lactulose - MIX0-461': {
        'Add': set(['CPD-3561[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'D11 - Sucrose - MIX0-462': {
        'Add': set(['SUCROSE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'D12 - Uridine - MIX0-463': {
        'Add': set(['URIDINE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'E1 - L-Glutamine - MIX0-464': {
        'Add': set(['GLN[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),        
    },
    'E2 - M-Tartaric acid - MIX0-465': None, # not in the model
    'E3 - D-Glucose- 1-Phosphate - MIX0-466': {
        'Add': set(['GLC-1-P[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'E4 - D-Fructose- 6-Phosphate - MIX0-467': {
        'Add': set(['FRUCTOSE-6P[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'E5 - Tween 80 - MIX0-799': None, # not in the model
    'E6 - α-Hydroxy glutaric_acid γ-Lactone - MIX0-793': {
        'Add': set(['CPD-13414[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'E7 - α-Hydroxy butyric_acid - MIX0-790': {
        'Add': set(['CPD-3564[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'E8 - β-Methyl-D- Glucoside - MIX0-784': {
        'Add': set(['CPD-3570[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'E9 - Adonitol - MIX0-468': {
        'Add': set(['RIBITOL[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'E10 - Maltotriose - MIX0-469': {
        'Add': set(['MALTOTRIOSE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'E11 - 2-Deoxyadenosine - MIX0-470': {
        'Add': set(['DEOXYADENOSINE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'E12 - Adenosine - MIX0-471': {
        'Add': set(['ADENOSINE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'F1 - Gly-Asp - MIX0-778': {
        'Add': set(['CPD-13406[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'F2 - Citric acid - MIX0-472': {
        'Add': set(['CIT[p]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'F3 - M-Inositol  - MIX0-473': {
        'Add': set(['MYO-INOSITOL[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    }, # lacks uptake and metabolic pathway
    'F4 - D-Threonine - MIX0-474': {
        'Add': set(['D-THREONINE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'F5 - Fumaric acid - MIX0-475': {
        'Add': set(['FUM[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'F6 - Bromosuccinic acid - MIX0-779': None, # not in the model
    'F7 - Propionic acid - MIX0-476': {
        'Add': set(['PROPIONATE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'F8 - Mucic acid - MIX0-477': {
        'Add': set(['D-GALACTARATE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'F9 - Glycolic acid - MIX0-478': {
        'Add': set(['GLYCOLLATE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'F10 - Glyoxylic acid - MIX0-479': {
        'Add': set(['GLYOX[p]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'F11 - D-Cellobiose - MIX0-480': {
        'Add': set(['CELLOBIOSE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'F12 - Inosine - MIX0-481': {
        'Add': set(['INOSINE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'G1 - Gly-Glu - MIX0-482': {
        'Add': set(['CPD-3569[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'G2 - Tricarballylic acid - MIX0-483': None, # not in the model
    'G3 - L-Serine - MIX0-484': {
        'Add': set(['SER[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'G4 - L-Threonine - MIX0-485': {
        'Add': set(['THR[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'G5 - L-Alanine - MIX0-486': {
        'Add': set(['L-ALPHA-ALANINE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'G6 - Ala-Gly - MIX0-772': {
        'Add': set(['ALA-GLY[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'G7 - Acetoacetic acid - MIX0-487': {
        'Add': set(['3-KETOBUTYRATE[p]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'G8 - N-Acetyl-D Mannosamine* - MIX0-488': {
        'Add': set(['N-acetyl-D-mannosamine[p]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'G9 - Mono- Methylsuccinate - MIX0-489': None, # not in the model
    'G10 - Methyl pyruvate - MIX0-490': None, # not in the model
    'G11 - D-Malic acid - MIX0-491': {
        'Add': set(['CPD-660[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'G12 - L-Malic acid - MIX0-492': {
        'Add': set(['MAL[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'H1 - Gly-Pro - MIX0-493': {
        'Add': set(['CPD-10814[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'H2 - p-Hydroxy phenyl Acetic_acid - MIX0-494': {
        'Add': set(['4-HYDROXYPHENYLACETATE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    }, # lacks uptake and metabolic pathway
    'H3 - m-Hydroxy phenyl Acetic_acid - MIX0-495': {
        'Add': set(['3-HYDROXYPHENYLACETATE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    }, # lacks uptake and metabolic pathway
    'H4 - Tyramine - MIX0-496': {
        'Add': set(['TYRAMINE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    }, # lacks uptake and metabolic pathway
    'H5 - D-Psicose - MIX0-497': {
        'Add': set(['Ket0-D-Psicose[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    }, # lacks uptake and metabolic pathway
    'H6 - L-Lyxose - MIX0-498': {
        'Add': set(['L-LYXOSE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'H7 - Glucuronamide - MIX0-499': None, # not in the model
    'H8 - Pyrunic acid - MIX0-500': {
        'Add': set(['PYRUVATE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'H9 - L-Galactonic acid-γ- Lactone - MIX0-794': {
        'Add': set(['CPD-330[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'H10 - D-Galacturonic acid* - MIX0-501': {
        'Add': set(['CPD-15633[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'H11 - Phenyl ethylamine - MIX0-502': {
        'Add': set(['PHENYLETHYLAMINE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
    'H12 - 2-Aminoethanol - MIX0-503': {
        'Add': set(['ETHANOL-AMINE[e]']),
        'Remove': set(['GLC[p]', 'CA+2[p]']),
    },
}

In [10]:
# run tests
condition_names = []
cp3_oofv = dict({})
plateID_to_condition = dict({})
for condition_name, condition in conditions.items():
    
    # store in dictionary the optimal objective function value
    temp = condition_name.split(' - ')
    plate_ID = temp[0]
    carbon_source = temp[1]
    plateID_to_condition[plate_ID] = carbon_source
    
    # solve the cvxpy problem
    objective_weights = {'secretion': 0.01, 'efficiency': 0.000001, 'kinetics': 0.0000001, 'diversity': 0.00001}
    if condition == None:
        cp3_oofv[plate_ID] = None
        continue
    oofv, solution_flux, test_reaction_names, S_new, test_metabolites, test_kinetic = test_NetworkFlowModel(
                                            objective_weights,
                                            uptake_addition=condition['Add'], uptake_removal=condition['Remove'],
                                            solver_choice=cp.MOSEK)    
    
    # get the fluxes
    sim_flux = pd.DataFrame({f'sim_cp3_{condition_name}': solution_flux}, index = test_reaction_names)
    condition_names.append(f'sim_cp3_{condition_name}')
    df_all = pd.concat([df_all, sim_flux], axis=1)
    cp3_oofv[plate_ID] = oofv
    
    print(f"""Finished enviornment: {condition_name} with objective function value: {oofv}""")

/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18617: UserWarning: Argument sub in putvarboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putvarboundlist: Incorrect array format causing data to be copied");
/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18925: UserWarning: Argument subj in putclist: Incorrect array format causing data to be copied
  warnings.warn("Argument subj in putclist: Incorrect array format causing data to be copied");
/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18349: UserWarning: Argument sub in putconboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putconboundlist: Incorrect array format causing data to be copied");


Finished enviornment: A1 - Carbon Negative Control - MIX0-80 with objective function value: 42274.6323620587


/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18617: UserWarning: Argument sub in putvarboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putvarboundlist: Incorrect array format causing data to be copied");
/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18925: UserWarning: Argument subj in putclist: Incorrect array format causing data to be copied
  warnings.warn("Argument subj in putclist: Incorrect array format causing data to be copied");
/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18349: UserWarning: Argument sub in putconboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putconboundlist: Incorrect array format causing data to be copied");


Finished enviornment: A2 - L-Arabinose* - MIX0-420 with objective function value: 19623.081178307115


/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18617: UserWarning: Argument sub in putvarboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putvarboundlist: Incorrect array format causing data to be copied");
/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18925: UserWarning: Argument subj in putclist: Incorrect array format causing data to be copied
  warnings.warn("Argument subj in putclist: Incorrect array format causing data to be copied");
/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18349: UserWarning: Argument sub in putconboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putconboundlist: Incorrect array format causing data to be copied");


...

Finished enviornment: H10 - D-Galacturonic acid* - MIX0-501 with objective function value: 20657.653586377095


/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18617: UserWarning: Argument sub in putvarboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putvarboundlist: Incorrect array format causing data to be copied");
/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18925: UserWarning: Argument subj in putclist: Incorrect array format causing data to be copied
  warnings.warn("Argument subj in putclist: Incorrect array format causing data to be copied");
/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18349: UserWarning: Argument sub in putconboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putconboundlist: Incorrect array format causing data to be copied");


Finished enviornment: H11 - Phenyl ethylamine - MIX0-502 with objective function value: 20987.204445446892


/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18617: UserWarning: Argument sub in putvarboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putvarboundlist: Incorrect array format causing data to be copied");
/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18925: UserWarning: Argument subj in putclist: Incorrect array format causing data to be copied
  warnings.warn("Argument subj in putclist: Incorrect array format causing data to be copied");
/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18349: UserWarning: Argument sub in putconboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putconboundlist: Incorrect array format causing data to be copied");


Finished enviornment: H12 - 2-Aminoethanol - MIX0-503 with objective function value: 21860.708874119267


In [11]:
df_all['is_new'] = 'Old Reactions'
df_all.loc[ADDED_RXNS, 'is_new'] = 'Heena\'s Reactions'
df_all.loc[fba_new_reaction_ids, 'is_new'] = 'New Reactions'

df_all['kinetic'] = [kinetic[r] if r in kinetic_reaction_ids else False for r in df_all.index]

In [12]:
df_all

,0,sim_cp3_A1 - Carbon Negative Control - MIX0-80,sim_cp3_A2 - L-Arabinose* - MIX0-420,sim_cp3_A3 - N-Acetyl-D- Glucosamine* - MIX0-421,sim_cp3_A4 - D-Saccharic acid - MIX0-422,sim_cp3_A5 - Succinic acid - MIX0-423,sim_cp3_A6 - D-Galactose* - MIX0-424,sim_cp3_A7 - L-Aspartic acid - MIX0-425,sim_cp3_A8 - L-Proline - MIX0-426,sim_cp3_A9 - D-Alanine - MIX0-427,...,sim_cp3_H4 - Tyramine - MIX0-496,sim_cp3_H5 - D-Psicose - MIX0-497,sim_cp3_H6 - L-Lyxose - MIX0-498,sim_cp3_H8 - Pyrunic acid - MIX0-500,sim_cp3_H9 - L-Galactonic acid-γ- Lactone - MIX0-794,sim_cp3_H10 - D-Galacturonic acid* - MIX0-501,sim_cp3_H11 - Phenyl ethylamine - MIX0-502,sim_cp3_H12 - 2-Aminoethanol - MIX0-503,is_new,kinetic
1-ACYLGLYCEROL-3-P-ACYLTRANSFER-RXN,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Old Reactions,False
1.1.1.127-RXN,0.000000e+00,1.0,1.0,1.000000e+00,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Old Reactions,False
1.1.1.127-RXN (reverse),0.000000e+00,1.0,1.0,1.000000e+00,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Old Reactions,False
1.1.1.215-RXN (reverse),0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Old Reactions,False
1.1.1.251-RXN,0.000000e+00,1.0,1.0,1.000000e+00,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Old Reactions,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XYLULOKIN-RXN-CPD-24961/ATP//XYLULOSE-5-PHOSPHATE/ADP/PROTON.47.,0.000000e+00,-0.0,-0.0,-3.943785e-18,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,Old Reactions,False
YIAE1-RXN (reverse),0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Old Reactions,False
glycogen-monomer-extension,9.005122e+03,-0.0,8662.0,8.662000e+03,-0.0,-0.0,8662.0,-0.0,8662.0,-0.0,...,-0.0,-0.0,8662.0,-0.0,8662.0,-0.0,8662.0,-0.0,Old Reactions,False
DISULFOXRED-RXN[CCO-PERI-BAC]-MONOMER0-4152/MONOMER0-4438//MONOMER0-4438/MONOMER0-4152.71.DEPHOSICITDEHASE-RXN,0.000000e+00,1.0,1.0,1.000000e+00,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Old Reactions,False


In [13]:
%store -r df_gene_usage genes_to_enzymes df_all_gene_usage new_genes

In [14]:
conditions_previous = ['basal', 'acetate', 'rich', 'anaerobic_basal', 'anaerobic_acetate', 'anaerobic_rich']
conditions_all = conditions_previous + condition_names

df_all_gene_usage = df_all_gene_usage.copy()

gene_used_before = df_gene_usage['is_used']
all_gene_used_before = df_all_gene_usage['is_used']

df_all_gene_usage = df_all_gene_usage.drop(['is_used'], axis = 1)
df_all_gene_usage_cp3 = df_all_gene_usage.copy()
reaction_catalysts = metabolism.parameters["reaction_catalysts"]

for condition in condition_names:
    df_all_gene_usage_cp3[condition] = 0
    for rxn in reaction_names:
        is_used = df_all.loc[rxn, condition]
        enzymes = reaction_catalysts.get(rxn,[])
        for enz in enzymes:
            enz = enz[:-3]
            genes = get_keys(genes_to_enzymes, enz)
            for gene in genes:
                if is_used:
                    df_all_gene_usage_cp3.loc[gene, condition] += 1
                if rxn in kinetic_reaction_ids:
                    df_all_gene_usage_cp3.loc[gene, 'has_kinetic'] = True


# tally usage
df_all_gene_usage_cp3['is_used'] = np.any(df_all_gene_usage_cp3[conditions_all] > 0, axis=1)
df_gene_usage_cp3 = df_all_gene_usage_cp3.loc[new_genes,:]

In [15]:
# previous % new gene usage
num_genes = len(gene_used_before)
assert len(gene_used_before) == len(df_gene_usage_cp3['is_used']), 'Different number of new genes'

perc_gene_usage_prev = sum(gene_used_before)/num_genes
perc_gene_usage_new  = sum(df_gene_usage_cp3['is_used'])/num_genes

print(f'% new genes usage before checkpoints is: {perc_gene_usage_prev: 0.2%}')
print(f'          % new genes usage added through microarray 1 is: {perc_gene_usage_new: 0.2%}')

% new genes usage before checkpoints is:  4.56%
          % new genes usage added through microarray 1 is:  30.29%


# Add checkpoint 2 reactions and condition testings

In [16]:
conditions = {
    'G7408-α-D-fructopyranose': {
        'Add': set(['CPD-10725[e]']),
        'Remove': set(['GLC[p]']),
    },
    'G7408-β-D-fructopyranose': {
        'Add': set(['CPD-10726[e]']),
        'Remove': set(['GLC[p]']),
    },
    'G7408-keto-D-fructose':{
        'Add': set(['CPD-15382[e]']),
        'Remove': set(['GLC[p]']),
    },
    'EG11724-KO_EG11691':{
        'Add': set(['ADENINE[e]']),
        'Remove': set([]),
        'Remove Reaction': ['TRANS-RXN0-577']
    },
    'EG10816-α-D-ribopyranose':{
        'Add': set(['CPD-15829[e]']),
        'Remove': set(['GLC[p]'])
    },
    'EG10818-aldehydo-D-ribose':{
        'Add': set(['CPD-15818[e]']),
        'Remove': set(['GLC[p]'])
    },
    'EG11055-1-O-methyl-β-D-glucuronate':{
        'Add': set(['CPD-3611[e]']),
        'Remove': set(['GLC[p]'])
    },
    'EG12393-ADD_TARTRATE[p]':{
        'Add': set(['TARTRATE[p]']),
        'Remove': set([])
    },
    'EG11054-α-D-mannopyranose 6-phosphate':{
        'Add': set(['MANNOSE-6P[e]']),
        'Remove': set(['GLC[p]'])
    },
    'EG10172-Oxidative_Stress':{
        'Add': set(['SUPER-OXIDE[e]']),
        'Remove': set([])
    },
    'EG10780-Thiosulfate':{
        'Add': set(['S2O3[e]','HCN[e]']),
        'Remove': set(['SULFATE[p]'])
    },
    'Isoprimeverose': {
        'Add': set(['CPD0-1202[e]']),
        'Remove': set(['GLC[p]']),
    },
    'Oxamate': {
        'Add': set(['S-ALLANTOIN[e]']),
        'Remove': set(['AMMONIUM[c]', 'OXYGEN-MOLECULE[p]']),
    },
    '3-HYDROXYPHENYL-PROPIONATE':{
        'Add': set(['3-HYDROXYPHENYL-PROPIONATE[e]']),
        'Remove': set(['GLC[p]'])
    },
    '3-PHENYLPROPIONATE':{
        'Add': set(['3-PHENYLPROPIONATE[e]']),
        'Remove': set(['GLC[p]'])
    },
    'cinnate':{
        'Add': set(['CPD-10797[e]', 'CPD-674[e]']),
        'Remove': set(['GLC[p]'])
    },
    'L-galactonate':{
        'Add': set(['CPD0-1083[e]']),
        'Remove': set(['GLC[p]'])
    },
}

In [17]:
condition_names = []
for condition_name, condition in conditions.items():
    
    if 'Remove Reaction' in condition:
        remove_reaction = condition['Remove Reaction']
    else:
        remove_reaction = None
    
    objective_weights = {'secretion': 0.01, 'efficiency': 0.000001, 'kinetics': 0.0000001, 'diversity': 0.00001}
    
    _, solution_flux, test_reaction_names, S_new, test_metabolites, test_kinetic = test_NetworkFlowModel(
                                            objective_weights,
                                            uptake_addition=condition['Add'], uptake_removal=condition['Remove'],
                                            remove_reaction=remove_reaction, solver_choice=cp.MOSEK
    )    
    
    # get the fluxes
    sim_flux = pd.DataFrame({f'sim_cp2_{condition_name}': solution_flux}, index = test_reaction_names)
    condition_names.append(f'sim_cp2_{condition_name}')
    df_all = pd.concat([df_all, sim_flux], axis=1)
    
    print(f"""Finished enviornment: {condition_name}""")

/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18617: UserWarning: Argument sub in putvarboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putvarboundlist: Incorrect array format causing data to be copied");
/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18925: UserWarning: Argument subj in putclist: Incorrect array format causing data to be copied
  warnings.warn("Argument subj in putclist: Incorrect array format causing data to be copied");
/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18349: UserWarning: Argument sub in putconboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putconboundlist: Incorrect array format causing data to be copied");


Finished enviornment: G7408-α-D-fructopyranose


/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18617: UserWarning: Argument sub in putvarboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putvarboundlist: Incorrect array format causing data to be copied");
/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18925: UserWarning: Argument subj in putclist: Incorrect array format causing data to be copied
  warnings.warn("Argument subj in putclist: Incorrect array format causing data to be copied");
/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18349: UserWarning: Argument sub in putconboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putconboundlist: Incorrect array format causing data to be copied");


Finished enviornment: G7408-β-D-fructopyranose


/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18617: UserWarning: Argument sub in putvarboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putvarboundlist: Incorrect array format causing data to be copied");
/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18925: UserWarning: Argument subj in putclist: Incorrect array format causing data to be copied
  warnings.warn("Argument subj in putclist: Incorrect array format causing data to be copied");
/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18349: UserWarning: Argument sub in putconboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putconboundlist: Incorrect array format causing data to be copied");


...

Finished enviornment: 3-PHENYLPROPIONATE


/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18617: UserWarning: Argument sub in putvarboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putvarboundlist: Incorrect array format causing data to be copied");
/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18925: UserWarning: Argument subj in putclist: Incorrect array format causing data to be copied
  warnings.warn("Argument subj in putclist: Incorrect array format causing data to be copied");
/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18349: UserWarning: Argument sub in putconboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putconboundlist: Incorrect array format causing data to be copied");


Finished enviornment: cinnate


/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18617: UserWarning: Argument sub in putvarboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putvarboundlist: Incorrect array format causing data to be copied");
/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18925: UserWarning: Argument subj in putclist: Incorrect array format causing data to be copied
  warnings.warn("Argument subj in putclist: Incorrect array format causing data to be copied");
/Users/HeenaSaqib/.pyenv/versions/3.11.3/envs/viv-ecoli/lib/python3.11/site-packages/mosek/__init__.py:18349: UserWarning: Argument sub in putconboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putconboundlist: Incorrect array format causing data to be copied");


Finished enviornment: L-galactonate


# calculate % gene usage

In [18]:
df_all_gene_usage_cp3_w_cp2 = df_all_gene_usage_cp3.copy()
reaction_catalysts = metabolism.parameters["reaction_catalysts"]
conditions_all = conditions_all + condition_names
for condition in condition_names:
    df_all_gene_usage_cp3_w_cp2[condition] = 0
    for rxn in reaction_names:
        is_used = df_all.loc[rxn, condition]
        enzymes = reaction_catalysts.get(rxn,[])
        for enz in enzymes:
            enz = enz[:-3]
            genes = get_keys(genes_to_enzymes, enz)
            for gene in genes:
                if is_used:
                    df_all_gene_usage_cp3_w_cp2.loc[gene, condition] += 1
                if rxn in kinetic_reaction_ids:
                    df_all_gene_usage_cp3_w_cp2.loc[gene, 'has_kinetic'] = True


# tally usage
df_all_gene_usage_cp3_w_cp2['is_used'] = np.any(df_all_gene_usage_cp3_w_cp2[conditions_all] > 0, axis=1)
df_gene_usage_cp3_w_cp2 = df_all_gene_usage_cp3_w_cp2.loc[new_genes,:]

In [19]:
# previous % new gene usage
num_genes = len(gene_used_before)
assert len(gene_used_before) == len(df_gene_usage_cp3_w_cp2['is_used']), 'Different number of new genes'

perc_gene_usage_prev = sum(gene_used_before)/num_genes
perc_gene_usage_cp3_w_cp2  = sum(df_gene_usage_cp3_w_cp2['is_used'])/num_genes

print(f'% new genes usage before checkpoint 1 and 2 is: {perc_gene_usage_prev: 0.2%}')
print(f'         %new genes enabled by microarray 1 is: {perc_gene_usage_new: 0.2%}')
print(f'          % new genes usage at checkpoint 3 is: {perc_gene_usage_cp3_w_cp2: 0.2%}')

% new genes usage before checkpoint 1 and 2 is:  4.56%
         %new genes enabled by microarray 1 is:  30.29%
          % new genes usage at checkpoint 3 is:  32.90%


In [20]:
%store -r df_all_gene_usage
# % all gene usage
num_genes = len(df_all_gene_usage)
perc_all_gene_usage_basal = sum(df_all_gene_usage_cp3_w_cp2.basal > 0)/num_genes * 100
perc_all_gene_usage_basic_conditions = sum(df_all_gene_usage['is_used'])/num_genes * 100
perc_all_gene_usage_cp3_w_cp2  = sum(df_all_gene_usage_cp3_w_cp2['is_used'])/num_genes * 100

perc_all = [perc_all_gene_usage_basal, perc_all_gene_usage_basic_conditions, perc_all_gene_usage_cp3_w_cp2]
perc_all

[28.468323977546113, 36.56776263031275, 62.55012028869287]

# Plot 1: Gene Usage Histogram at checkpoint 3 with different solvers and diversity objective

In [21]:
%store -r df_all_gene_usage_basal_with_diversity df_all_gene_usage_cp3_MOSEK df_all_gene_usage_cp3_GLOP 
%store -r perc_gene_usage_w_MOSEK perc_all_gene_usage_w_MOSEK

In [22]:
# % new gene usage
df_gene_usage = df_all_gene_usage.loc[new_genes,:]
df_gene_usage_basal_with_diversity = df_all_gene_usage_basal_with_diversity.loc[new_genes,:]
df_gene_usage_cp3_GLOP = df_all_gene_usage_cp3_GLOP.loc[new_genes,:]
df_gene_usage_cp3_MOSEK = df_all_gene_usage_cp3_MOSEK.loc[new_genes,:]
df_gene_usage_cp3_w_diversity = df_all_gene_usage_cp3_w_cp2.loc[new_genes,:].copy()

num_genes = len(df_gene_usage) 
perc_gene_usage_basal = sum(df_gene_usage.basal > 0)/num_genes * 100
perc_gene_usage_basal_w_diversity  = sum(df_gene_usage_basal_with_diversity['is_used'])/num_genes * 100
perc_gene_usage_cp3_GLOP  = sum(df_gene_usage_cp3_GLOP['is_used'])/num_genes * 100
perc_gene_usage_cp3_MOSEK  = sum(df_gene_usage_cp3_MOSEK['is_used'])/num_genes * 100
perc_gene_usage_cp3_w_diversity  = sum(df_gene_usage_cp3_w_diversity['is_used'])/num_genes * 100

perc = [perc_gene_usage_basal, perc_gene_usage_basal_w_diversity, perc_gene_usage_w_MOSEK,
        perc_gene_usage_cp3_GLOP, perc_gene_usage_cp3_w_diversity, perc_gene_usage_cp3_MOSEK] 
x_label = ['Basal', 'Basal with Diversity Objective', 'Basal with MOSEK', 'Cp3 (µArray) using GLOP', 
           'Cp3 ((µArray)) with Diversity Objective (MOSEK)', 'Cp3 (µArray) using MOSEK', ]

In [23]:
# % all gene usage
num_genes = len(df_all_gene_usage)
perc_all_gene_usage_basal = sum(df_all_gene_usage.basal > 0)/num_genes * 100
perc_all_gene_usage_basal_w_diversity = sum(df_all_gene_usage_basal_with_diversity['is_used'])/num_genes * 100
perc_all_gene_usage_cp3_GLOP  = sum(df_all_gene_usage_cp3_GLOP['is_used'])/num_genes * 100
perc_all_gene_usage_cp3_MOSEK  = sum(df_all_gene_usage_cp3_MOSEK['is_used'])/num_genes * 100
perc_all_gene_usage_cp3_w_diversity  = sum(df_all_gene_usage_cp3_w_cp2['is_used'])/num_genes * 100

perc_all = [perc_all_gene_usage_basal, perc_all_gene_usage_basal_w_diversity, perc_all_gene_usage_w_MOSEK,
            perc_all_gene_usage_cp3_GLOP, perc_all_gene_usage_cp3_w_diversity, perc_all_gene_usage_cp3_MOSEK] 

In [24]:
# Create a DataFrame
df = pd.DataFrame({
    "Condition": x_label,
    "New Metabolic Gene": perc,
    "All Metabolic Gene": perc_all,
})

# Reshape to long format
df_long = df.melt(id_vars="Condition", 
                  value_vars=["New Metabolic Gene", "All Metabolic Gene"],
                  var_name="Gene Group",
                  value_name="Percent Usage")

# Create grouped bar plot
fig = px.bar(
    df_long,
    x="Condition",
    y="Percent Usage",
    color="Gene Group",              # creates a legend
    barmode="group",                 # side-by-side bars
    text="Percent Usage",
    title="New vs All Metabolic Gene Usage by Condition",
    labels={"Percent Usage": "Percent gene usage (%)"},
    color_discrete_map={
        "New Metabolic Gene": "#4C78A8",
        "All Metabolic Gene": "#c26426"
    }
)

# Customize appearance
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',
    yaxis_title="Percent gene usage (%)",
    yaxis=dict(range=[0, 80]),
    xaxis_title=None,
    xaxis_tickangle=-35,
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    margin=dict(t=50, b=50, l=50, r=50),
    width=800,   # width in pixels
    height=500
)

fig.show(renderer='browser')
# save 
# fig.write_image("notebooks/Heena notebooks/Metabolism_New Genes/out/gene_usage_histogram_cp3.png", scale=5, width=800, height=500)

Plot 2: Create growth no growth heatmap

In [27]:
def discrete_colorscale(bvals, colors):
    """
    bvals - list of values bounding intervals/ranges of interest
    colors - list of rgb or hex colorcodes for values in [bvals[k], bvals[k+1]],0<=k < len(bvals)-1
    returns the plotly  discrete colorscale
    """
    if len(bvals) != len(colors)+1:
        raise ValueError('len(boundary values) should be equal to  len(colors)+1')
    bvals = sorted(bvals)     
    nvals = [(v-bvals[0])/(bvals[-1]-bvals[0]) for v in bvals]  #normalized values
    
    dcolorscale = [] #discrete colorscale
    for k in range(len(colors)):
        dcolorscale.extend([[nvals[k], colors[k]], [nvals[k+1], colors[k]]])
    return dcolorscale  

In [36]:
bvals = [14000, 25500, 40000, 123357053]
colors = ['#dda0dd', '#ffb6c2', '#fcf3bf']
dcolorsc = discrete_colorscale(bvals, colors)
dcolorsc

[[0.0, '#dda0dd'],
 [9.323589549871122e-05, '#dda0dd'],
 [9.323589549871122e-05, '#ffb6c2'],
 [0.0002107941985188254, '#ffb6c2'],
 [0.0002107941985188254, '#fcf3bf'],
 [1.0, '#fcf3bf']]

In [37]:
# Plate dimensions
rows = list("ABCDEFGH")
cols = list(range(1, 13))

# Create a 2D array with None as default
matrix = []
label = []
for r in rows:
    row_data = []
    row_text = []
    for c in cols:
        key = f"{r}{c}"
        row_data.append(cp3_oofv.get(key))  # Fill missing wells with None
        text = plateID_to_condition.get(key)
        text_segment = text.split(' ')
        text = '<br>'.join(text_segment)
        row_text.append(text)
    matrix.append(row_data)
    label.append(row_text)

In [38]:
import plotly.graph_objects as go
heatmap = go.Heatmap(z=matrix, 
                     x=[str(c) for c in cols],
                     y=rows,
                     text=label,
                     texttemplate="%{text}",
                     textfont={"size": 10},
                     colorscale = dcolorsc, 
                     colorbar = dict(thickness=25, 
                                     tickvals=tickvals, 
                                     ticktext=ticktext),
                     )

fig = go.Figure(data=[heatmap])
fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',
    title="Plate Reader Heatmap with Values",
    xaxis_title="Column",
    yaxis_title="Row",
    yaxis_autorange='reversed'  
)

fig.show(renderer='browser')
# fig.write_image("notebooks/Heena notebooks/Metabolism_New Genes/out/heatmap_microarray1_C_source.png", scale=5, width=1200, height=800)

In [26]:
sum(df_all_gene_usage_cp3_MOSEK['is_used'])

920

In [35]:
cp3_oofv

{'A1': 42274.6323620587,
 'A2': 19623.081178307115,
 'A3': 17373.65734705962,
 'A4': 30066.117644743437,
 'A5': 32220.10201556705,
 'A6': 19581.949292868296,
 'A7': 38847.00378168431,
 'A8': 16155.81852157957,
 'A9': 19918.355371619244,
 'A10': 19374.888642803762,
 'A11': 19374.894913557928,
 'A12': 17972.326913141897,
 'B1': 27116.002581882945,
 'B2': 17862.43229017058,
 'B3': 15530.339466120902,
 'B4': 19699.349394604866,
 'B5': 20657.691890159746,
 'B6': 17580.854611526527,
 'B7': 22160.876146565002,
 'B8': 19622.982511041453,
 'B9': 23316.493710145533,
 'B10': 42274.62996257616,
 'B11': 17862.452396560395,
 'B12': 23603.05181058393,
 'C1': 21385.910751279822,
 'C2': 22269.635286485012,
 'C3': 36642.51522322408,
 'C4': 20401.313287130302,
 'C5': None,
 'C6': 19700.423348254837,
 'C7': 19374.521671166174,
 'C8': 29792.08203667421,
 'C9': 19374.870504769187,
 'C10': 18975.569076226267,
 'C11': 19478.057337120026,
 'C12': 21446.695922063474,
 'D1': 41377.21748487919,
 'D2': 42274.63391

In [41]:
df_all_cp3_w_diversity = df_all_gene_usage_cp3_w_cp2.copy()
cp3_w_diversity_oofv = cp3_oofv.copy()

In [42]:
%store df_all_cp3_w_diversity cp3_w_diversity_oofv 

Stored 'df_all_cp3_w_diversity' (DataFrame)
Stored 'cp3_w_diversity_oofv' (dict)


In [43]:
df_all_cp3_w_diversity

,Enzyme encoded,Reactions,has_kinetic,basal,acetate,rich,anaerobic_basal,anaerobic_acetate,anaerobic_rich,sim_cp3_A1 - Carbon Negative Control - MIX0-80,...,sim_cp2_EG12393-ADD_TARTRATE[p],sim_cp2_EG11054-α-D-mannopyranose 6-phosphate,sim_cp2_EG10172-Oxidative_Stress,sim_cp2_EG10780-Thiosulfate,sim_cp2_Isoprimeverose,sim_cp2_Oxamate,sim_cp2_3-HYDROXYPHENYL-PROPIONATE,sim_cp2_3-PHENYLPROPIONATE,sim_cp2_cinnate,sim_cp2_L-galactonate
Gene ID (EcoCyc),,,,,,,,,,,,,,,,,,,,,
EG11679,AAS-MONOMER,"[ACYLGPEACYLTRANS-RXN, RXN-5741, RXN0-5513]",False,0,0,0,0,0,0,2,...,2,2,2,2,2,2,2,2,2,2
EG11647,ACETYL-COA-CARBOXYLTRANSFER-CPLX,[RXN0-5055],False,1,1,1,1,0,0,1,...,1,1,1,1,1,1,1,1,1,1
EG10276,BIOTIN-CARBOXYL-CPLX,[BIOTIN-CARBOXYL-RXN],True,1,1,1,1,0,0,1,...,1,1,1,1,1,1,1,1,1,1
EG10217,ACETYL-COA-CARBOXYLTRANSFER-CPLX,[RXN0-5055],False,1,1,1,1,0,0,1,...,1,1,1,1,1,1,1,1,1,1
EG10022,ISOCIT-LYASE,[ISOCIT-CLEAV-RXN],False,1,1,0,0,1,0,2,...,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EG12215,YHHO-MONOMER,"[RXN0-5205, 3.6.3.3-RXN, TRANS-RXN-452]",False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
G7017,ABC-63-CPLX,[ABC-63-RXN],False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
G7018,ABC-63-CPLX,[ABC-63-RXN],False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
num_genes = len(df_gene_usage) 
perc_gene_usage_basal = sum(df_gene_usage.basal > 0)/num_genes * 100
perc_gene_usage_basal_w_diversity  = sum(df_gene_usage_basal_with_diversity['is_used'])/num_genes * 100
perc_gene_usage_cp3_GLOP  = sum(df_gene_usage_cp3_GLOP['is_used'])/num_genes * 100
perc_gene_usage_cp3_MOSEK  = sum(df_gene_usage_cp3_MOSEK['is_used'])/num_genes * 100
perc_gene_usage_cp3_w_diversity  = sum(df_gene_usage_cp3_w_diversity['is_used'])/num_genes * 100

In [47]:
df_gene_usage_cp3_MOSEK['is_used']

Gene ID (EcoCyc)
EG10022     True
EG10023     True
EG11942     True
EG11724     True
EG12462    False
           ...  
G7248      False
G7408       True
EG12517    False
EG12518    False
EG12520    False
Name: is_used, Length: 307, dtype: bool

['1-ACYLGLYCEROL-3-P-ACYLTRANSFER-RXN',
 '1.1.1.127-RXN',
 '1.1.1.127-RXN (reverse)',
 '1.1.1.215-RXN (reverse)',
 '1.1.1.251-RXN',
 '1.1.1.251-RXN (reverse)',
 '1.1.1.271-RXN (reverse)',
 '1.1.1.274-RXN (reverse)',
 '1.1.1.283-RXN (reverse)',
 '1.1.1.39-RXN',
 '1.1.1.83-RXN',
 '1.11.1.15-RXN',
 '1.13.11.16-RXN',
 '1.3.1.9-RXN (reverse)',
 '1.5.1.20-RXN-5-METHYL-THF/NAD//METHYLENE-THF/NADH/PROTON.44.',
 '1.5.1.20-RXN-5-METHYL-THF/NAD//METHYLENE-THF/NADH/PROTON.44. (reverse)',
 '1.5.1.20-RXN-5-METHYL-THF/NADP//METHYLENE-THF/NADPH/PROTON.46.',
 '1.5.1.20-RXN-5-METHYL-THF/NADP//METHYLENE-THF/NADPH/PROTON.46. (reverse)',
 '1.5.1.20-RXN-CPD-1302/NAD//CPD-12996/NADH/PROTON.36.',
 '1.5.1.20-RXN-CPD-1302/NAD//CPD-12996/NADH/PROTON.36. (reverse)',
 '1.5.1.20-RXN-CPD-1302/NADP//CPD-12996/NADPH/PROTON.38.',
 '1.5.1.20-RXN-CPD-1302/NADP//CPD-12996/NADPH/PROTON.38. (reverse)',
 '1.7.2.2-RXN-A-REDUCED-TORY-PROTEIN/NITRITE/PROTON//AMMONIUM/an-oxidized-TorY-protein/WATER.79.',
 '1.7.2.2-RXN-Cytochrome

{'1-ACYLGLYCEROL-3-P-ACYLTRANSFER-RXN': '1-ACYLGLYCEROL-3-P-ACYLTRANSFER-RXN',
 '1.1.1.127-RXN': '1.1.1.127-RXN',
 '1.1.1.127-RXN (reverse)': '1.1.1.127-RXN',
 '1.1.1.215-RXN (reverse)': '1.1.1.215-RXN',
 '1.1.1.251-RXN': '1.1.1.251-RXN',
 '1.1.1.251-RXN (reverse)': '1.1.1.251-RXN',
 '1.1.1.271-RXN (reverse)': '1.1.1.271-RXN',
 '1.1.1.274-RXN (reverse)': '1.1.1.274-RXN',
 '1.1.1.283-RXN (reverse)': '1.1.1.283-RXN',
 '1.1.1.39-RXN': '1.1.1.39-RXN',
 '1.1.1.83-RXN': '1.1.1.83-RXN',
 '1.11.1.15-RXN': '1.11.1.15-RXN',
 '1.13.11.16-RXN': '1.13.11.16-RXN',
 '1.3.1.9-RXN (reverse)': '1.3.1.9-RXN',
 '1.5.1.20-RXN-5-METHYL-THF/NAD//METHYLENE-THF/NADH/PROTON.44.': '1.5.1.20-RXN',
 '1.5.1.20-RXN-5-METHYL-THF/NAD//METHYLENE-THF/NADH/PROTON.44. (reverse)': '1.5.1.20-RXN',
 '1.5.1.20-RXN-5-METHYL-THF/NADP//METHYLENE-THF/NADPH/PROTON.46.': '1.5.1.20-RXN',
 '1.5.1.20-RXN-5-METHYL-THF/NADP//METHYLENE-THF/NADPH/PROTON.46. (reverse)': '1.5.1.20-RXN',
 '1.5.1.20-RXN-CPD-1302/NAD//CPD-12996/NADH/PROTON.36.